In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

x_train = x_train[..., tf.newaxis]   
x_test = x_test[..., tf.newaxis]  

In [ ]:
x_train[0].shape

In [ ]:
x_test[0].shape

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(1000).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self):
    super().__init__()
    self.conv1 = Conv2D(3, 3, activation="relu")
    self.flatten = Flatten()
    self.d1 = Dense(128, activation="relu")
    self.d2 = Dense(10)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    x = self.d2(x)
    return x

model = MyModel()

In [ ]:
loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimiser = tf.keras.optimizers.Adam()

In [ ]:

train_loss = tf.keras.metrics.Mean()
test_loss = tf.keras.metrics.Mean()

train_accuracy = tf.keras.metrics.SparseCategoricalCrossentropy()
test_accuracy = tf.keras.metrics.SparseCategoricalCrossentropy()

In [ ]:
@tf.function
def train(images, labels):
  with tf.GradientTape() as tape:
    pred = model(images)
    loss = loss_function(labels, pred)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimiser.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, pred)


In [ ]:
@tf.function
def test(images, labels):
  pred = model(images)
  loss = loss_function(labels, pred)

  test_loss(loss)
  test_accuracy(labels, pred)

In [ ]:
epochs = 5

for epoch in range(epochs):
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
  #train
    for images, labels in train_dataset:
      train(images, labels)

    #test
    for images, labels in test_dataset:
      test(images, labels)


    print("epoch:", epoch + 1,
          f"Train loss: {train_loss.result()}",
          f"Train accuracy: {train_accuracy.result()}",
          f"Test loss: {test_loss.result()}",
          f"test accuracy: {test_accuracy.result()}")



In [ ]:
import cv2
import numpy as np

img = cv2.imread("/content/seven.webp")
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img = cv2.resize(img, (28, 28))
img = img[..., tf.newaxis]
img = img[tf.newaxis, ...]
img = img.astype("float32")

pred = model(img)
result = np.argmax(pred)
result

In [ ]:
model.save("My_Model")

In [ ]:
model = tf.keras.models.load_model("/content/My_Model")